<a href="https://colab.research.google.com/github/LoosonWu/geek_homework/blob/main/stacking_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

!git clone https://github.com/LoosonWu/data.git


Cloning into 'data'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [3]:
import lightgbm as lgb  
import pandas as pd  
import numpy as np  
import pickle  
from sklearn.metrics import roc_auc_score  
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score 
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.datasets.samples_generator import make_blobs


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [107]:
def load_data():
  train_data = pd.read_csv('./data/train_final.csv', engine='python')
  train_y = train_data.iloc[:,15]
  train_x = train_data.iloc[:,1:15].drop(['continuous_annual_inc_joint','continuous_dti_joint'],1)
  train_x = train_x.fillna(0)
  test_data = pd.read_csv('./data/test_final.csv', engine='python')
  test_y = test_data.iloc[:,15]
  test_x = test_data.iloc[:,1:15].drop(['continuous_annual_inc_joint','continuous_dti_joint'],1)
  test_x = test_x.fillna(0)
  return train_x, train_y, test_x, test_y

In [108]:
print("Loading Data ... ")  
train_x, train_y, test_x, test_y = load_data()  
data, target = make_blobs(n_samples=50000, centers=2, random_state=0, cluster_std=0.60)

Loading Data ... 


In [41]:
clfs = [RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5)]

In [9]:
'''切分一部分数据作为测试集'''
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.33, random_state=2017)
X_predict

array([[1.55201968, 0.58723382],
       [0.97477516, 4.73585524],
       [0.79459967, 3.57899822],
       ...,
       [3.78308507, 1.51634693],
       [0.36107361, 4.36264542],
       [1.65024072, 5.64267771]])

In [15]:
print(enumerate(skf.split(X,y)))

In [113]:
def stacking(train_x, train_y, x_predict, y_predict, n_folds):
  '''采用2级stacking'''
  dataset_blend_train = np.zeros((train_x.shape[0], len(clfs)))
  dataset_blend_test = np.zeros((train_y.shape[0], len(clfs)))
  skf = StratifiedKFold(n_splits=n_folds)
  '''第一级stacking里有5个model'''
  for j, clf in enumerate(clfs):
      dataset_blend_test_j = np.zeros((train_y.shape[0], n_folds))
      for i, (train, test) in enumerate(skf.split(train_x,train_y)):
          print('i=',i)
          X_train, y_train, X_test, y_test = train_x[train], train_y[train], train_x[test], train_y[test]
          clf.fit(X_train, y_train)
          y_submission = clf.predict_proba(X_test)[:, 1]
          dataset_blend_train[test, j] = y_submission
          '''把每一个model的每一次split train起来的model用在测试数据集上测试后的结构放到dataset_blend_test_j的一个column里'''
          dataset_blend_test_j[:, i] = clf.predict_proba(x_predict)[:, 1]
      '''将一个model跑完后的预测结构求平均值作为这个model的结果'''
      dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
      print("val auc Score: %f" % roc_auc_score(y_predict, dataset_blend_test[:, j]))
  '''第二级stacking里有1个model'''
  clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
  '''将第一级stacking的结构用来拟合'''
  clf.fit(dataset_blend_train, train_y)
  '''将测试数据集在第一级stacking里的预测值用来作为第二级stacking模型的测试级'''
  y_submission = clf.predict_proba(dataset_blend_test)[:, 1]
  # y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
  print("val auc Score: %f" % (roc_auc_score(y_predict, y_submission)))

In [114]:
np.set_printoptions(suppress=True)
npa_train_x = np.array(train_x) 
npa_train_y = np.array(train_y) 
npa_test_x = np.array(test_x) 
npa_test_y = np.array(test_y)
stacking(npa_train_x, npa_train_y, npa_test_x, npa_test_y, 5)

i= 0
i= 1
i= 2
i= 3
i= 4
val auc Score: 0.945832
i= 0
i= 1
i= 2
i= 3
i= 4
val auc Score: 0.946251
i= 0
i= 1
i= 2
i= 3
i= 4
val auc Score: 0.944945
i= 0
i= 1
i= 2
i= 3
i= 4
val auc Score: 0.944718
i= 0
i= 1
i= 2
i= 3
i= 4
val auc Score: 0.955457
blend result
val auc Score: 0.955120


In [91]:
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.33, random_state=2017)


dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))

'''5折stacking'''
n_folds = 5
skf=StratifiedKFold(n_splits=n_folds)
for j, clf in enumerate(clfs):
    '''依次训练各个单模型'''
    # print(j, clf)
    dataset_blend_test_j = np.zeros((X_predict.shape[0], n_folds))
    for i, (train, test) in enumerate(skf.split(X,y)):
        '''使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。'''
        # print("Fold", i)
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        print(X_train)
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    '''对于测试集，直接用这k个模型的预测值均值作为新的特征。'''
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print("val auc Score: %f" % roc_auc_score(y_predict, dataset_blend_test[:, j]))
# clf = LogisticRegression()
clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]

print("Linear stretch of predictions to [0,1]")
y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())
print("blend result")
print("val auc Score: %f" % (roc_auc_score(y_predict, y_submission)))

[[2.5928463  1.06655543]
 [1.37507144 1.65843695]
 [2.24065241 0.68977891]
 ...
 [0.69947609 4.59298804]
 [1.57949104 0.04266078]
 [1.02000175 4.29567311]]
[[0.20849521 2.97833057]
 [0.10909607 4.20652405]
 [1.72761745 0.81885791]
 ...
 [0.69947609 4.59298804]
 [1.57949104 0.04266078]
 [1.02000175 4.29567311]]
[[0.20849521 2.97833057]
 [0.10909607 4.20652405]
 [1.72761745 0.81885791]
 ...
 [0.69947609 4.59298804]
 [1.57949104 0.04266078]
 [1.02000175 4.29567311]]
[[0.20849521 2.97833057]
 [0.10909607 4.20652405]
 [1.72761745 0.81885791]
 ...
 [0.69947609 4.59298804]
 [1.57949104 0.04266078]
 [1.02000175 4.29567311]]
[[0.20849521 2.97833057]
 [0.10909607 4.20652405]
 [1.72761745 0.81885791]
 ...
 [1.01308683 4.89818154]
 [1.20312737 4.61039211]
 [0.03560861 4.66253541]]
val auc Score: 0.999564
[[2.5928463  1.06655543]
 [1.37507144 1.65843695]
 [2.24065241 0.68977891]
 ...
 [0.69947609 4.59298804]
 [1.57949104 0.04266078]
 [1.02000175 4.29567311]]
[[0.20849521 2.97833057]
 [0.10909607 4.

In [73]:
X=np.array([
    [1,2,3,4],
    [11,12,13,14],
    [21,22,23,24],
    [31,32,33,34],
    [41,42,43,44],
    [51,52,53,54],
    [61,62,63,64],
    [71,72,73,74]
])

y=np.array([1,1,0,0,1,1,0,0])
#n_folds这个参数没有，引入的包不同，
sfolder = StratifiedKFold(n_splits=4,random_state=0,shuffle=False)

for train, test in sfolder.split(X,y):
    print('Train: %s | test: %s' % (train, test))
    print(X[train])
    print(y[train])


Train: [1 3 4 5 6 7] | test: [0 2]
[[11 12 13 14]
 [31 32 33 34]
 [41 42 43 44]
 [51 52 53 54]
 [61 62 63 64]
 [71 72 73 74]]
[1 0 1 1 0 0]
Train: [0 2 4 5 6 7] | test: [1 3]
[[ 1  2  3  4]
 [21 22 23 24]
 [41 42 43 44]
 [51 52 53 54]
 [61 62 63 64]
 [71 72 73 74]]
[1 0 1 1 0 0]
Train: [0 1 2 3 5 7] | test: [4 6]
[[ 1  2  3  4]
 [11 12 13 14]
 [21 22 23 24]
 [31 32 33 34]
 [51 52 53 54]
 [71 72 73 74]]
[1 1 0 0 1 0]
Train: [0 1 2 3 4 6] | test: [5 7]
[[ 1  2  3  4]
 [11 12 13 14]
 [21 22 23 24]
 [31 32 33 34]
 [41 42 43 44]
 [61 62 63 64]]
[1 1 0 0 1 0]


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [66]:
train_x

,continuous_delinq_2yrs,continuous_dti,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt
0,0.0,5.91,679.0,675.0,3600.0,3600.0,1.0,123.03,13.99,564.0,560.0,3600.0
1,1.0,16.06,719.0,715.0,24700.0,24700.0,4.0,820.28,11.99,699.0,695.0,24700.0
2,0.0,10.78,699.0,695.0,20000.0,20000.0,0.0,432.66,10.78,704.0,700.0,20000.0
3,1.0,25.37,699.0,695.0,10400.0,10400.0,3.0,289.91,22.45,704.0,700.0,10400.0
4,0.0,10.20,694.0,690.0,11950.0,11950.0,0.0,405.18,13.44,759.0,755.0,11950.0
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,30.60,674.0,670.0,16000.0,16000.0,0.0,544.29,13.67,684.0,680.0,16000.0
49996,0.0,19.23,789.0,785.0,8500.0,8500.0,0.0,259.52,6.24,774.0,770.0,8500.0
49997,0.0,9.06,674.0,670.0,20050.0,20050.0,0.0,682.06,13.67,689.0,685.0,20050.0
49998,0.0,7.39,694.0,690.0,35000.0,35000.0,3.0,1129.19,9.99,819.0,815.0,35000.0


In [65]:
test_x

,continuous_delinq_2yrs,continuous_dti,continuous_fico_range_high,continuous_fico_range_low,continuous_funded_amnt,continuous_funded_amnt_inv,continuous_inq_last_6mths,continuous_installment,continuous_int_rate,continuous_last_fico_range_high,continuous_last_fico_range_low,continuous_loan_amnt
0,1.0,25.65,664.0,660.0,13000.0,13000.0,1.0,428.63,11.49,659.0,655.0,13000.0
1,0.0,22.01,729.0,725.0,11000.0,11000.0,0.0,339.10,6.89,769.0,765.0,11000.0
2,0.0,26.39,764.0,760.0,10000.0,10000.0,1.0,301.15,5.32,684.0,680.0,10000.0
3,0.0,12.26,664.0,660.0,13800.0,13800.0,0.0,318.75,13.67,724.0,720.0,13800.0
4,0.0,17.55,674.0,670.0,13550.0,13550.0,3.0,491.23,18.20,574.0,570.0,13550.0
...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,25.93,709.0,705.0,18500.0,18500.0,0.0,402.15,10.99,734.0,730.0,18500.0
49996,0.0,29.86,664.0,660.0,12000.0,12000.0,1.0,268.70,12.29,524.0,520.0,12000.0
49997,0.0,6.56,679.0,675.0,22000.0,22000.0,0.0,709.78,9.99,669.0,665.0,22000.0
49998,0.0,20.35,739.0,735.0,15000.0,15000.0,0.0,462.41,6.89,719.0,715.0,15000.0


In [ ]:
# create dataset for lightgbm  
lgb_train = lgb.Dataset(train_x, train_y)  
lgb_test = lgb.Dataset(test_x, test_y, reference=lgb_train)  

In [ ]:
# specify configurations as a dict  
params = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,'lambda_l1': 1,'is_unbalance': True} 


In [ ]:
# train  
print('Start training...')  
gbm = lgb.train(params,lgb_train,num_boost_round=10000,valid_sets=lgb_test,early_stopping_rounds=1000)  
print('Finished training...')  

Start training...
[1]	valid_0's binary_logloss: 0.490906	valid_0's auc: 0.953783
Training until validation scores don't improve for 1000 rounds.
[2]	valid_0's binary_logloss: 0.487574	valid_0's auc: 0.955215
[3]	valid_0's binary_logloss: 0.484313	valid_0's auc: 0.955072
[4]	valid_0's binary_logloss: 0.48112	valid_0's auc: 0.955141
[5]	valid_0's binary_logloss: 0.477994	valid_0's auc: 0.954987
[6]	valid_0's binary_logloss: 0.474926	valid_0's auc: 0.954881
[7]	valid_0's binary_logloss: 0.471919	valid_0's auc: 0.954968
[8]	valid_0's binary_logloss: 0.468975	valid_0's auc: 0.954788
[9]	valid_0's binary_logloss: 0.466083	valid_0's auc: 0.954806
[10]	valid_0's binary_logloss: 0.463248	valid_0's auc: 0.954781
[11]	valid_0's binary_logloss: 0.460461	valid_0's auc: 0.955142
[12]	valid_0's binary_logloss: 0.457719	valid_0's auc: 0.955536
[13]	valid_0's binary_logloss: 0.455027	valid_0's auc: 0.955646
[14]	valid_0's binary_logloss: 0.452389	valid_0's auc: 0.955705
[15]	valid_0's binary_logloss: 0

In [ ]:
preds = gbm.predict(test_x, num_iteration=gbm.best_iteration) 
size = np.size(preds)
print(size)

50000


In [ ]:
threshold = 0.5  
for i in range(0,size):
    if preds[i] >= threshold:
       preds[i] = 1
    else:  
       preds[i] = 0

In [ ]:
accuracy = accuracy_score(test_y,preds) 
print(accuracy)

0.90242


# **Reports**:

# Round 1:只选取前面15个feature

> **params** = params = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 16,'max_depth': 4,'learning_rate': 0.01,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 5,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.89608

# Round 2: 在同一个learning rate里调整叶子树和深度，叶子树保证是深度的2^max_depth

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 64,'max_depth': 6,'learning_rate': 0.01,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 5,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.89868

# Round 3: 在同一个叶子树和深度，调整learning rate到0.05，结果是正确率降低了

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 64,'max_depth': 6,'learning_rate': 0.05,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 5,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.89788

# Round 4: 基于前两轮的经验，降低learning rate到0.005，结果是正确率相对上一轮提高了，但不是目前最高

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 64,'max_depth': 6,'learning_rate': 0.005,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 5,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.89822

# Round 5: 如果保证learning rate是0.005，把叶子树和深度提高，得到了目前最高准确率，总体感觉是叶子树越多，learning rate同时越低，带来的拟合度越好

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 5,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.90102

# Round 6: 基于Round 5的结果，我将bagging_freq调成1，结果拟合度提高了一点点，我的理解是，这样改成了每轮迭代都做bagging，对于每轮来讲都是公平的，虽然损失了效率

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,'feature_fraction': 0.9,'bagging_fraction': 0.95,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.90122

# Round 7: 基于Round 6的结果，去掉feature_fraction，让它用上所有feature，损失了效率，但拟合度提高了一点点

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.95,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.9021

# Round 8: 基于Round 7的结果，去掉bagging，泛化降低

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.95,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.9017

# Round 9: 基于Round 8的结果，降低bagging比例到0.9，提高泛化

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.9,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.902

# Round 10: 基于Round 7的结果，bagging比例调到0.98，降低泛化，但拟合度降低，有可能0.95是最优选

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.98,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.90174

# Round 11: 基于前面调bagging的结果，试下bagging比例调到0.955，效果也没0.95的时候好

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.955,'bagging_freq': 1,'lambda_l1': 1,'lambda_l2': 0.001,'is_unbalance': True}

> **Accuracy**: 0.90204

# Round 12: 基于前面调bagging的结果，试下bagging比例调到0.95，同时去掉lambda_l2

> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.955,'bagging_freq': 1,'lambda_l1': 1,'is_unbalance': True}

> **Accuracy**: 0.9022

# **Round 13: 基于Round 12，尝试调lambda_l1=0.5**
> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.955,'bagging_freq': 1,'lambda_l1': 0.5,'is_unbalance': True}

> **Accuracy**: 0.90178

# **Round 14: 基于Round 13，尝试调lambda_l1=2**
> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','bagging_fraction': 0.955,'bagging_freq': 1,'lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.90178

# **Round 15: 尝试用goss**
> **params** = {'boosting_type': 'goss','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.9002

# **Round 16: 尝试用goss，learning_rate改0.0001**
> **params** = {'boosting_type': 'goss','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.0001,','lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.80452

# **Round 17: 尝试用goss，learning_rate改0.01**
> **params** = {'boosting_type': 'goss','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.0001,','lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.9003

# **Round 18: 尝试用gbdt，learning_rate改0.01，选中所有feature，事实证明并不是有更多feature拟合就更好**
> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.01,','lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.90232

# **Round 19: 尝试用gbdt，learning_rate改0.005，选中所有feature，事实证明并不是有更多feature拟合就更好**
> **params** = {'boosting_type': 'gbdt','objective': 'binary','metric': {'binary_logloss', 'auc'},'num_leaves': 256,'max_depth': 8,'learning_rate': 0.005,','lambda_l1': 2,'is_unbalance': True}

> **Accuracy**: 0.90242